In [ ]:
import pandas as pd
import sqlite3
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import warnings

warnings.filterwarnings('ignore')  

# 配置中文显示
matplotlib.rcParams['font.sans-serif'] = ['Microsoft YaHei', 'SimHei', 'Arial Unicode MS']
matplotlib.rcParams['axes.unicode_minus'] = False

# 配置路径
data_path = r'D:\Postgraduate\工作\stage 1\archive\data.csv'
save_dir = r"D:\Postgraduate\工作\stage 1\archive\results"
os.makedirs(save_dir, exist_ok=True)  # 确保目录存在

print("🧹 开始数据清洗...")

# 1. 读取数据
df = pd.read_csv(data_path, encoding='latin-1')

# 2. 转换日期格式
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# 3. 添加衍生字段
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day
df['Hour'] = df['InvoiceDate'].dt.hour
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek
df['Date'] = df['InvoiceDate'].dt.date

# 4. 计算总金额
df['TotalAmount'] = df['Quantity'] * df['UnitPrice']

# 5. 识别退货订单
df['IsReturn'] = df['InvoiceNo'].str.startswith('C')

# 6. 数据清洗规则
df_clean = df[
    (df['Quantity'] > 0) &
    (df['UnitPrice'] > 0) &
    (df['CustomerID'].notna()) &
    (df['TotalAmount'] < 10000)
].copy()

print(f"✅ 原始数据: {len(df):,} 条")
print(f"✅ 清洗后数据: {len(df_clean):,} 条")
print(f"✅ 清洗掉: {len(df) - len(df_clean):,} 条 ({(len(df) - len(df_clean))/len(df)*100:.1f}%)")

# ==================== 创建SQLite数据库 ====================
print("\n📦 创建SQLite数据库...")
conn = sqlite3.connect('ecommerce.db')
df_clean.to_sql('transactions', conn, index=False, if_exists='replace')

print(f"✅ 数据库创建成功！表名: transactions, 记录数: {len(df_clean):,} 条")

# ==================== 数据验证====================
print("\n" + "="*60)
print("🔍 数据验证".center(60))
print("="*60)

result1 = pd.read_sql_query("""
    SELECT 
        COUNT(*) as 总订单数,
        COUNT(DISTINCT InvoiceNo) as 订单号数量,
        COUNT(DISTINCT CustomerID) as 客户数量,
        COUNT(DISTINCT StockCode) as 商品数量,
        COUNT(DISTINCT Country) as 国家数量,
        ROUND(SUM(TotalAmount), 2) as 总GMV
    FROM transactions
""", conn)
print("\n📊 整体概览:")
print(result1.to_string(index=False))

result2 = pd.read_sql_query("""
    SELECT 
        MIN(InvoiceDate) as 最早日期,
        MAX(InvoiceDate) as 最晚日期,
        COUNT(DISTINCT Date) as 总天数
    FROM transactions
""", conn)
print("\n⏰ 时间范围:")
print(result2.to_string(index=False))

result3 = pd.read_sql_query("""
    SELECT 
        Country as 国家,
        COUNT(*) as 订单数,
        ROUND(SUM(TotalAmount), 2) as GMV,
        COUNT(DISTINCT CustomerID) as 客户数
    FROM transactions
    GROUP BY Country
    ORDER BY GMV DESC
    LIMIT 5
""", conn)
print("\n🌍 Top5国家:")
print(result3.to_string(index=False))

print("\n" + "="*60)
print("✅ 数据库准备完成")
print("="*60)